Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [41]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

logging.basicConfig(level=logging.DEBUG)

In [66]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [43]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [67]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

## Second Greedy Algorithm - Farthest Insertion

In [ ]:
def farthest_insertion_tsp():
    # Start with a small tour between the first two cities
    tour = [0, 1, 0]
    unvisited = set(range(2, len(CITIES)))

    # Continue until all cities are added
    while unvisited:
        # Find the unvisited city farthest from any city in the current tour
        farthest_city, farthest_dist = None, 0
        for city in unvisited:
            min_dist_to_tour = min(DIST_MATRIX[city, t_city] for t_city in tour)
            if min_dist_to_tour > farthest_dist:
                farthest_city, farthest_dist = city, min_dist_to_tour

        # Find the optimal position to insert the farthest city
        best_position, best_cost = None, float('inf')
        for i in range(1, len(tour)):
            cost = (
                DIST_MATRIX[tour[i - 1], farthest_city]
                + DIST_MATRIX[farthest_city, tour[i]]
                - DIST_MATRIX[tour[i - 1], tour[i]]
            )
            if cost < best_cost:
                best_position, best_cost = i, cost

        # Insert the city into the tour and log the step
        tour.insert(best_position, farthest_city)
        logging.debug(
            f"step: {CITIES.at[tour[best_position-1],'name']} -> "
            f"{CITIES.at[farthest_city,'name']} -> {CITIES.at[tour[best_position+1],'name']} "
            f"({DIST_MATRIX[tour[best_position-1], farthest_city]:.2f}km + "
            f"{DIST_MATRIX[farthest_city, tour[best_position+1]]:.2f}km)"
        )
        unvisited.remove(farthest_city)

    # Log the return to the starting city
    logging.debug(
        f"step: {CITIES.at[tour[-2],'name']} -> {CITIES.at[tour[0],'name']} ({DIST_MATRIX[tour[-2], tour[0]]:.2f}km)"
    )

    # Calculate and log the final result
    total_cost = tsp_cost(tour)
    logging.info(f"result: Found a path of {len(tour)-1} steps, total length {total_cost:.2f} km")

    return tour, total_cost

# Run the updated farthest insertion TSP
farthest_insertion_tsp()


DEBUG:root:step: Ancona -> Cagliari -> Andria (609.70km + 651.14km)
DEBUG:root:step: Ancona -> Turin -> Cagliari (492.26km + 661.37km)
DEBUG:root:step: Cagliari -> Syracuse -> Andria (592.60km + 469.85km)
DEBUG:root:step: Ancona -> Bolzano -> Turin (364.04km + 326.05km)
DEBUG:root:step: Turin -> Leghorn -> Cagliari (269.41km + 491.24km)
DEBUG:root:step: Andria -> Latina -> Ancona (285.77km + 241.85km)
DEBUG:root:step: Ancona -> Trieste -> Bolzano (228.81km + 209.68km)
DEBUG:root:step: Cagliari -> Palermo -> Syracuse (390.28km + 206.47km)
DEBUG:root:step: Ancona -> Ferrara -> Trieste (204.43km + 191.98km)
DEBUG:root:step: Leghorn -> Sassari -> Cagliari (344.66km + 173.89km)
DEBUG:root:step: Bolzano -> Bergamo -> Turin (157.42km + 170.39km)
DEBUG:root:step: Andria -> Salerno -> Latina (141.80km + 180.71km)
DEBUG:root:step: Latina -> Pescara -> Ancona (155.12km + 139.24km)
DEBUG:root:step: Syracuse -> Messina -> Andria (126.41km + 343.44km)
DEBUG:root:step: Latina -> Terni -> Pescara (123

([0,
  33,
  12,
  30,
  9,
  23,
  43,
  41,
  5,
  40,
  45,
  44,
  6,
  3,
  20,
  18,
  22,
  42,
  13,
  28,
  25,
  32,
  19,
  4,
  29,
  10,
  16,
  36,
  7,
  24,
  8,
  37,
  31,
  17,
  38,
  2,
  1,
  11,
  35,
  21,
  14,
  15,
  34,
  39,
  26,
  27,
  0],
 np.float64(4172.762613916411))

The Farthest insertion finds a better solution, with 4172km (vs 4436.03km of the simple greedy). The algorithm iteratively adds the city that is farthest from any city in the current tour. After selecting the farthest city, it determines the best position to insert this city into the tour to minimize the increase in total distance.

## Evolutionary Algorithm

In [45]:
import random

# Parameters
POPULATION_SIZE = 1000
MUTATION_RATE = 0.2
GENERATIONS = 1000

def create_random_tour():
    tour = list(range(1, len(CITIES)))  # Exclude the first city initially
    random.shuffle(tour)
    tour = [0] + tour + [0]  # Add the starting city at the beginning and end
    return tour

def fitness(tour):
    return tsp_cost(tour)

def select_parents(population):
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    tournament.sort(key=fitness)
    return tournament[0], tournament[1]

def crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last city (returning to the start)
    start, end = sorted(random.sample(range(1, size), 2))  # Start and end should be between 1 and size-1
    
    # Create the offspring with None as placeholders
    child = [None] * (size + 1)  # Size + 1 for the returning city

    # Copy the segment from parent1
    child[start:end] = parent1[start:end]

    # Fill the remaining positions with genes from parent2
    p2_index = 1  # Start after the first city
    for i in range(1, size + 1):
        if child[i] is None:
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]
            p2_index += 1
            if p2_index >= len(parent2) - 1:  # Prevent index out of range
                p2_index = 1  # Reset to avoid going out of bounds

    child[0] = 0  # Ensure the first city is the starting city
    child[-1] = 0  # Ensure the last city returns to the starting city

    return child

def mutate(tour):
    if len(tour) <= 3:  # If there are only 3 cities, mutation is not needed
        return
    for swapped in range(1, len(tour) - 1):  # Don't mutate the first and last city
        if random.random() < MUTATION_RATE:
            swap_with = random.randint(1, len(tour) - 2)  # Ensure swapping within valid range
            tour[swapped], tour[swap_with] = tour[swap_with], tour[swapped]


In [46]:
def evolutionary_algorithm():
    # Initialize population with random tours
    population = [create_random_tour() for _ in range(POPULATION_SIZE)]
    
    for generation in range(GENERATIONS):
        # Evaluate the fitness of the population
        population.sort(key=fitness)
        
        # Log the best fitness for this generation
        best_tour = population[0]
        best_cost = fitness(best_tour)
        #logging.info(f"Generation {generation}: Best cost = {best_cost:.2f}")

        # Create a new population
        new_population = []

        # Elitism: Keep the best individual
        new_population.append(best_tour)

        # Generate the next generation
        while len(new_population) < POPULATION_SIZE:
            # Select parents
            parent1, parent2 = select_parents(population)
            # Perform crossover
            child = crossover(parent1, parent2)
            # Perform mutation
            mutate(child)
            # Ensure the child is a valid tour (start and end at the same city)
            child[0] = 0  # Ensure the first city is the starting city
            child[-1] = 0  # Ensure the last city returns to the starting city
            # Add the new child to the new population
            new_population.append(child)

        # Replace the old population with the new one
        population = new_population

    # Final evaluation
    population.sort(key=fitness)
    best_tour = population[0]
    best_cost = fitness(best_tour)

    # Logging the final result in the specified format
    logging.info(f"result: Found a path of {len(best_tour)-1} steps, total length {best_cost:.2f} km")

    return best_tour, best_cost

# Run the evolutionary algorithm
best_tour, best_cost = evolutionary_algorithm()


INFO:root:result: Found a path of 46 steps, total length 11817.79 km


I tried POPULATION_SIZE = 100 MUTATION_RATE = 0.5 GENERATIONS = 200000, and after running for 38 minutes, it gave a solution of total lengh aroung 11000km, so not really good. From previous testing it seems like the problem is the mutation rate, which is way too high. after trying with a lower mutation rate I found a better solution much faster. POPULATION_SIZE = 1000, MUTATION_RATE = 0.1, GENERATIONS = 20000 found a solution of 8966, after half an hour. Much better but still inferior to the greedy approach. After this, I decided to keep the population size and mutation rate the same, but put a much higher limit of generations(200 000), and leave it running overnight. After 5 hours, the best solution is only 7190.12 km, still pretty far from the greedy solution. 

## Steady State EA

In [48]:
import random
import logging

POPULATION_SIZE = 100
MUTATION_RATE = 0.1
GENERATIONS = 50000
REPLACEMENT_RATE = 2 

def create_random_tour():
    tour = list(range(1, len(CITIES))) 
    random.shuffle(tour)
    tour = [0] + tour + [0] 
    return tour

def fitness(tour):
    return tsp_cost(tour)

def select_parents(population):
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    tournament.sort(key=fitness)
    return tournament[0], tournament[1]

def crossover(parent1, parent2):
    size = len(parent1) - 1
    start, end = sorted(random.sample(range(1, size), 2))
    child = [None] * (size + 1)

    child[start:end] = parent1[start:end]
    p2_index = 1
    for i in range(1, size + 1):
        if child[i] is None:
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]
            p2_index += 1

    child[0] = 0
    child[-1] = 0
    return child

def mutate(tour):
    if len(tour) <= 3:
        return
    for swapped in range(1, len(tour) - 1):
        if random.random() < MUTATION_RATE:
            swap_with = random.randint(1, len(tour) - 2)
            tour[swapped], tour[swap_with] = tour[swap_with], tour[swapped]

def steady_state_algorithm():
    population = [create_random_tour() for _ in range(POPULATION_SIZE)]
    population.sort(key=fitness)

    for generation in range(GENERATIONS):
        for _ in range(REPLACEMENT_RATE):
            parent1, parent2 = select_parents(population)
            child = crossover(parent1, parent2)
            mutate(child)
            child[0], child[-1] = 0, 0  
            child_cost = fitness(child)

            if child_cost < fitness(population[-1]):
                population[-1] = child
                population.sort(key=fitness) 

        best_tour = population[0]
        best_cost = fitness(best_tour)
        #logging.info(f"Generation {generation}: Best cost = {best_cost:.2f}")

    best_tour = population[0]
    best_cost = fitness(best_tour)
    logging.info(f"result: Found a path of {len(best_tour)-1} steps, total length {best_cost:.2f} km")

    return best_tour, best_cost

logging.basicConfig(level=logging.INFO)
best_tour, best_cost = steady_state_algorithm()


INFO:root:result: Found a path of 46 steps, total length 5606.80 km


The steady state does much better and is much faster. Only replacing a few individuals each generation, sorting the population by fitness at the end of each replacement seems to be the way to go. Running it for only 1 minute it already found a solution with a cost of around 5.6k. So I decided to set the number of generations to 100_000, and see if it could find a solution even better than the greedy approach. Letting it run for longer, it seems that the algorithm reaches a plateu and barely improve, fixating aroung a value of 5342km. I will now apply the algorithm to the other files. 

## China
### Greedy

In [63]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])

# Recreate the distance matrix for the new city data
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


In [54]:
initial_tou, initial_cos = farthest_insertion_tsp()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()